In [1]:
import json
import requests
import time
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv()) # retrieve all env vars

fmp_api_key = os.getenv("FMP_API_KEY")

client = OpenAI()



### Steps to create this assistant

1. Define financial `functions`

2. Define `Assistant`

3. Create a `Thread`

4. Add `Messages`

5. `Run` the Assistant

## 1. Functions


In [2]:
def get_income_statement(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_cash_flow_statement(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_key_metrics(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_ratios(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())    

def get_financial_growth(ticker: str, period: str, limit: int = 100):
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&limit={limit}&apikey={fmp_api_key}"
    response = requests.get(url)
    return json.dumps(response.json())      

In [3]:
# Storing functions in a dictionary

available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_financial_ratios,
    "get_financial_growth": get_financial_growth
}

## 2. Assistant

In [4]:
# def run_assistant(user_message: str):
assistant = client.beta.assistants.create(
    instructions="Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends",
    model="gpt-3.5-turbo-1106",
    tools=[
        # income statement function
        {
            "type": "function",
            "function": {
                    "name": "get_income_statement",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # balance sheet
        {
            "type": "function",
            "function": {
                    "name": "get_balance_sheet",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # cash flow statement
        {
            "type": "function",
            "function": {
                    "name": "get_cash_flow_statement",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # key metrics
        {
            "type": "function",
            "function": {
                    "name": "get_key_metrics",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # financial ratios
        {
            "type": "function",
            "function": {
                    "name": "get_financial_ratios",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },

        # financial growth
        {
            "type": "function",
            "function": {
                    "name": "get_financial_growth",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker": {"type": "string"},
                            "period": {"type": "string"},
                            "limit": {"type": "integer"}
                        }
                    }
            }
        },
    ]
)

dict(assistant)

{'id': 'asst_mCWswCy1i5NK5Rqi3VAa8fmE',
 'created_at': 1704909745,
 'description': None,
 'file_ids': [],
 'instructions': 'Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}, 'period': {'type': 'string'}, 'limit': {'type': 'integer'}}}, description=None), type='function'),
  ToolFunction(function=FunctionDefinition(name='get_balance_sheet', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}, 'period': {'type': 'string'}, 'limit': {'type': 'integer'}}}, description=None), type='function'),
  ToolFunction(function=FunctionDefinition(nam

## 3. Thread

In [5]:
thread = client.beta.threads.create()

dict(thread)

{'id': 'thread_WrKc7V84TpoUay03553mhwbw',
 'created_at': 1704909746,
 'metadata': {},
 'object': 'thread'}

## 4. Add Message

In [6]:
# Adding initial user message
user_message = "Give an overview of income statement"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

dict(message)


{'id': 'msg_xwkM0HO6JBruClCkc27chD1p',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Give an overview of income statement'), type='text')],
 'created_at': 1704909747,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_WrKc7V84TpoUay03553mhwbw'}

## 5. Run 

In [7]:
# 5.1 Start a Run

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

dict(run)


{'id': 'run_WosCEJmzlJtR1BnsUfSB2DHp',
 'assistant_id': 'asst_mCWswCy1i5NK5Rqi3VAa8fmE',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704909748,
 'expires_at': 1704910348,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_WrKc7V84TpoUay03553mhwbw',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}, 'period': {'type': 'string'}, 'limit': {'type': 'integer'}}}, description=None), type='function'),
  ToolAssistantToolsFunction(fu

In [9]:
# checking status

run_status = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

dict(run_status)

{'id': 'run_WosCEJmzlJtR1BnsUfSB2DHp',
 'assistant_id': 'asst_mCWswCy1i5NK5Rqi3VAa8fmE',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704909748,
 'expires_at': None,
 'failed_at': 1704909753,
 'file_ids': [],
 'instructions': 'Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends',
 'last_error': LastError(code='rate_limit_exceeded', message='You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'),
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1704909748,
 'status': 'failed',
 'thread_id': 'thread_WrKc7V84TpoUay03553mhwbw',
 'tools': [ToolAssistantToolsFunction(function=FunctionDef